# 📋 Notebook 02: Data Preparation

**Objective:** Prepare two versions of the dataset for modeling

**What we'll do:**
1. Load the clean data from Notebook 01
2. Create Dataset B (Full) - all 21 features
3. Create Dataset A (Clean) - remove potentially leaky features
4. Prepare scaling strategy
5. Save both datasets

**Why two datasets?**
- Dataset B (Full): Shows maximum predictive power (but might include target leakage)
- Dataset A (Clean): More realistic for preventive screening (removes consequences of diabetes)
- Comparing them demonstrates critical thinking about feature selection!

---

## 📦 Step 1: Imports and Setup

In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For scaling (we'll prepare the strategy, not fit yet)
from sklearn.preprocessing import StandardScaler

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports complete")

✅ Imports complete


## 📊 Step 2: Load Data from Notebook 01

In [2]:
# Load the dataset
df = pd.read_csv('C:\\Users\\yaros\\Desktop\\python\\faidm\\individual_project\\diabetes-classification-ml\\data\\CDC Diabetes Dataset.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

Dataset loaded successfully!
Shape: (253680, 22)

Columns: ['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']


In [3]:
# Comprehensive data quality check
print("=" * 60)
print("DATA QUALITY VERIFICATION")
print("=" * 60)

# 1. Basic checks
print("\n1️⃣ BASIC CHECKS")
print("-" * 60)
print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"Duplicate rows: {df.duplicated().sum()}")

# 2. Data types check
print("\n2️⃣ DATA TYPES")
print("-" * 60)
print(df.dtypes.value_counts())
print(f"\n⚠️ All columns should be numeric (float64 or int64)")

# 3. Check for any non-numeric values
print("\n3️⃣ NON-NUMERIC VALUES CHECK")
print("-" * 60)
non_numeric_cols = df.select_dtypes(exclude=['number']).columns.tolist()
if non_numeric_cols:
    print(f"⚠️ Non-numeric columns found: {non_numeric_cols}")
else:
    print("✅ All columns are numeric")

DATA QUALITY VERIFICATION

1️⃣ BASIC CHECKS
------------------------------------------------------------
Total rows: 253,680
Total columns: 22
Missing values: 0
Duplicate rows: 23899

2️⃣ DATA TYPES
------------------------------------------------------------
float64    22
Name: count, dtype: int64

⚠️ All columns should be numeric (float64 or int64)

3️⃣ NON-NUMERIC VALUES CHECK
------------------------------------------------------------
✅ All columns are numeric


In [4]:
# 4. Check value ranges for each feature
print("\n4️⃣ VALUE RANGE VERIFICATION")
print("-" * 60)
print("Checking if values are within expected ranges...\n")

# Expected ranges based on dataset description
expected_ranges = {
    'Diabetes_012': (0, 2),
    'HighBP': (0, 1),
    'HighChol': (0, 1),
    'CholCheck': (0, 1),
    'BMI': (12, 98),  # Reasonable human BMI range
    'Smoker': (0, 1),
    'Stroke': (0, 1),
    'HeartDiseaseorAttack': (0, 1),
    'PhysActivity': (0, 1),
    'Fruits': (0, 1),
    'Veggies': (0, 1),
    'HvyAlcoholConsump': (0, 1),
    'AnyHealthcare': (0, 1),
    'NoDocbcCost': (0, 1),
    'GenHlth': (1, 5),
    'MentHlth': (0, 30),
    'PhysHlth': (0, 30),
    'DiffWalk': (0, 1),
    'Sex': (0, 1),
    'Age': (1, 13),
    'Education': (1, 6),
    'Income': (1, 8)
}

range_issues = []

for col, (min_val, max_val) in expected_ranges.items():
    actual_min = df[col].min()
    actual_max = df[col].max()
    
    if actual_min < min_val or actual_max > max_val:
        range_issues.append(col)
        print(f"⚠️ {col}: Expected [{min_val}-{max_val}], Got [{actual_min}-{actual_max}]")

if not range_issues:
    print("✅ All features are within expected ranges")
else:
    print(f"\n⚠️ Found {len(range_issues)} features with unexpected ranges")


4️⃣ VALUE RANGE VERIFICATION
------------------------------------------------------------
Checking if values are within expected ranges...

✅ All features are within expected ranges


In [5]:
# 5. Check for outliers in continuous features
print("\n5️⃣ OUTLIER DETECTION (Continuous Features)")
print("-" * 60)

continuous_cols = ['BMI', 'MentHlth', 'PhysHlth']

for col in continuous_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_pct = (len(outliers) / len(df)) * 100
    
    print(f"\n{col}:")
    print(f"  Range: [{df[col].min():.1f} - {df[col].max():.1f}]")
    print(f"  Mean: {df[col].mean():.1f}, Median: {df[col].median():.1f}")
    print(f"  IQR bounds: [{lower_bound:.1f} - {upper_bound:.1f}]")
    print(f"  Outliers: {len(outliers):,} ({outlier_pct:.2f}%)")
    
    if outlier_pct > 5:
        print(f"  ⚠️ High percentage of outliers (>5%)")
    else:
        print(f"  ✅ Outlier percentage acceptable")


5️⃣ OUTLIER DETECTION (Continuous Features)
------------------------------------------------------------

BMI:
  Range: [12.0 - 98.0]
  Mean: 28.4, Median: 27.0
  IQR bounds: [13.5 - 41.5]
  Outliers: 9,847 (3.88%)
  ✅ Outlier percentage acceptable

MentHlth:
  Range: [0.0 - 30.0]
  Mean: 3.2, Median: 0.0
  IQR bounds: [-3.0 - 5.0]
  Outliers: 36,208 (14.27%)
  ⚠️ High percentage of outliers (>5%)

PhysHlth:
  Range: [0.0 - 30.0]
  Mean: 4.2, Median: 0.0
  IQR bounds: [-4.5 - 7.5]
  Outliers: 40,949 (16.14%)
  ⚠️ High percentage of outliers (>5%)


In [6]:
# 6. Check for unexpected value distributions in binary features
print("\n6️⃣ BINARY FEATURE DISTRIBUTION CHECK")
print("-" * 60)

binary_cols = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 
               'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
               'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 
               'DiffWalk', 'Sex']

print("\nChecking if binary features only contain 0 and 1...\n")

binary_issues = []
for col in binary_cols:
    unique_vals = df[col].unique()
    if not set(unique_vals).issubset({0.0, 1.0}):
        binary_issues.append(col)
        print(f"⚠️ {col}: Contains values other than 0/1: {unique_vals}")

if not binary_issues:
    print("✅ All binary features contain only 0 and 1")
else:
    print(f"\n⚠️ Found {len(binary_issues)} binary features with unexpected values")


6️⃣ BINARY FEATURE DISTRIBUTION CHECK
------------------------------------------------------------

Checking if binary features only contain 0 and 1...

✅ All binary features contain only 0 and 1


In [7]:
# 7. Target variable distribution check
print("\n7️⃣ TARGET VARIABLE CHECK")
print("-" * 60)

target_col = 'Diabetes_012'
target_counts = df[target_col].value_counts().sort_index()

print("\nClass distribution:")
for cls, count in target_counts.items():
    pct = (count / len(df)) * 100
    print(f"  Class {int(cls)}: {count:6,} ({pct:5.2f}%)")

# Calculate imbalance ratio
majority_class = target_counts.max()
minority_class = target_counts.min()
imbalance_ratio = majority_class / minority_class

print(f"\nImbalance ratio: {imbalance_ratio:.1f}:1")
if imbalance_ratio > 10:
    print("⚠️ SEVERE class imbalance detected (>10:1)")
    print("   → We'll need to handle this in modeling phase")
elif imbalance_ratio > 3:
    print("⚠️ Moderate class imbalance detected (>3:1)")
else:
    print("✅ Classes are relatively balanced")


7️⃣ TARGET VARIABLE CHECK
------------------------------------------------------------

Class distribution:
  Class 0: 213,703 (84.24%)
  Class 1:  4,631 ( 1.83%)
  Class 2: 35,346 (13.93%)

Imbalance ratio: 46.1:1
⚠️ SEVERE class imbalance detected (>10:1)
   → We'll need to handle this in modeling phase


In [8]:
# 8. Final summary
print("\n" + "=" * 60)
print("FINAL DATA QUALITY SUMMARY")
print("=" * 60)

quality_checks = {
    'No missing values': df.isnull().sum().sum() == 0,
    'No duplicates': df.duplicated().sum() == 0,
    'All numeric types': len(non_numeric_cols) == 0,
    'Values in expected ranges': len(range_issues) == 0,
    'Binary features valid': len(binary_issues) == 0,
}

print()
for check, passed in quality_checks.items():
    status = "✅" if passed else "⚠️"
    print(f"{status} {check}")

all_passed = all(quality_checks.values())

print("\n" + "=" * 60)
if all_passed:
    print("✅ ALL DATA QUALITY CHECKS PASSED!")
    print("✅ Dataset is ready for preprocessing and modeling")
else:
    print("⚠️ SOME ISSUES DETECTED - Review above for details")
    print("   (Note: Some issues like outliers may be expected)")
print("=" * 60)


FINAL DATA QUALITY SUMMARY

✅ No missing values
⚠️ No duplicates
✅ All numeric types
✅ Values in expected ranges
✅ Binary features valid

⚠️ SOME ISSUES DETECTED - Review above for details
   (Note: Some issues like outliers may be expected)


## 🧹 Step 3: Remove Duplicates FIRST

**Strategy:** Remove duplicates from the original dataset BEFORE creating Dataset A and Dataset B.

**Why this order?**
- Ensures both datasets have the same sample size
- Makes model comparison fair (only difference is features, not samples)
- Cleaner methodology

**Important:** After this step, we work with deduplicated data for ALL subsequent analysis.

In [9]:
print("=" * 60)
print("STEP 3: REMOVE DUPLICATES FROM ORIGINAL DATASET")
print("=" * 60)

# Count duplicates in original data
duplicates_original = df.duplicated().sum()
duplicate_pct = (duplicates_original / len(df)) * 100

print(f"\n📊 BEFORE deduplication:")
print(f"Total rows: {len(df):,}")
print(f"Duplicate rows: {duplicates_original:,} ({duplicate_pct:.2f}%)")
print(f"Unique rows: {len(df) - duplicates_original:,}")

STEP 3: REMOVE DUPLICATES FROM ORIGINAL DATASET

📊 BEFORE deduplication:
Total rows: 253,680
Duplicate rows: 23,899 (9.42%)
Unique rows: 229,781


In [10]:
# Remove duplicates and store in NEW variable
df_deduplicated = df.drop_duplicates().copy()

print(f"\n✅ AFTER deduplication:")
print(f"Before: {len(df):,} rows")
print(f"After:  {len(df_deduplicated):,} rows")
print(f"Removed: {len(df) - len(df_deduplicated):,} duplicate rows")

# Verify no duplicates remain
remaining_duplicates = df_deduplicated.duplicated().sum()
print(f"\n✅ Verification: {remaining_duplicates} duplicates remaining (should be 0)")


✅ AFTER deduplication:
Before: 253,680 rows
After:  229,781 rows
Removed: 23,899 duplicate rows

✅ Verification: 0 duplicates remaining (should be 0)


In [11]:
# Check class balance is preserved
print(f"\n📊 Class distribution AFTER deduplication:")
print("-" * 60)

for cls, count in df_deduplicated['Diabetes_012'].value_counts().sort_index().items():
    pct = (count / len(df_deduplicated)) * 100
    original_pct = (df['Diabetes_012'].value_counts().sort_index()[cls] / len(df)) * 100
    print(f"Class {int(cls)}: {count:6,} ({pct:5.2f}%) - Original: {original_pct:5.2f}%")

print("\n✅ Class proportions preserved after deduplication!")


📊 Class distribution AFTER deduplication:
------------------------------------------------------------
Class 0: 190,055 (82.71%) - Original: 84.24%
Class 1:  4,629 ( 2.01%) - Original:  1.83%
Class 2: 35,097 (15.27%) - Original: 13.93%

✅ Class proportions preserved after deduplication!


In [12]:
# FROM NOW ON, use df_deduplicated for ALL subsequent work
print("\n" + "=" * 60)
print("✅ DEDUPLICATED DATASET READY")
print("=" * 60)
print(f"\nWorking dataset: {len(df_deduplicated):,} rows, {len(df_deduplicated.columns)} columns")
print(f"\n⚠️ IMPORTANT: All subsequent steps will use df_deduplicated")


✅ DEDUPLICATED DATASET READY

Working dataset: 229,781 rows, 22 columns

⚠️ IMPORTANT: All subsequent steps will use df_deduplicated


## 🔍 Step 4: Identify Potentially Leaky Features

**Target Leakage** occurs when a feature is a *consequence* of the target variable, rather than a *cause*.

**Why this matters:**
- If we include leaky features, our model might look great in testing...
- But it won't work for **preventive screening** (before diabetes develops)
- It would only work for **diagnostic confirmation** (after symptoms appear)

**Potentially leaky features in this dataset:**

| Feature | Description | Why It Might Be Leaky |
|---------|-------------|----------------------|
| `DiffWalk` | Difficulty walking or climbing stairs | Often a **consequence** of diabetes (neuropathy, poor circulation) |
| `GenHlth` | Self-reported general health (1-5 scale) | People with diabetes naturally rate their health lower |
| `PhysHlth` | Days of poor physical health (0-30) | Similar to GenHlth - likely consequence of diabetes |

**Our strategy:**
1. Create **Dataset B (Full)** - keep all features (shows maximum predictive power)
2. Create **Dataset A (Clean)** - remove these 3 features (more realistic for prevention)
3. Compare model performance on both
4. Discuss implications in final report

In [13]:
# Define features to remove for clean dataset
potentially_leaky_features = ['DiffWalk', 'GenHlth', 'PhysHlth']

print("Potentially leaky features identified:")
for feature in potentially_leaky_features:
    print(f"  - {feature}")

print(f"\nThese will be removed in Dataset A (Clean)")

Potentially leaky features identified:
  - DiffWalk
  - GenHlth
  - PhysHlth

These will be removed in Dataset A (Clean)


## 🔍 Step 4: Identify Potentially Leaky Features

**Target Leakage** occurs when a feature is a *consequence* of the target variable, rather than a *cause*.

**Why this matters:**
- If we include leaky features, our model might look great in testing...
- But it won't work for **preventive screening** (before diabetes develops)
- It would only work for **diagnostic confirmation** (after symptoms appear)

**Potentially leaky features in this dataset:**

| Feature | Description | Why It Might Be Leaky |
|---------|-------------|----------------------|
| `DiffWalk` | Difficulty walking or climbing stairs | Often a **consequence** of diabetes (neuropathy, poor circulation) |
| `GenHlth` | Self-reported general health (1-5 scale) | People with diabetes naturally rate their health lower |
| `PhysHlth` | Days of poor physical health (0-30) | Similar to GenHlth - likely consequence of diabetes |

**Our strategy:**
1. Create **Dataset B (Full)** - keep all features (shows maximum predictive power)
2. Create **Dataset A (Clean)** - remove these 3 features (more realistic for prevention)
3. Compare model performance on both
4. Discuss implications in final report

In [14]:
# Define features to remove for clean dataset
potentially_leaky_features = ['DiffWalk', 'GenHlth', 'PhysHlth']

print("Potentially leaky features identified:")
for feature in potentially_leaky_features:
    print(f"  - {feature}")

print(f"\nThese will be removed in Dataset A (Clean)")

Potentially leaky features identified:
  - DiffWalk
  - GenHlth
  - PhysHlth

These will be removed in Dataset A (Clean)


## 📊 Step 5: Create Dataset B (Full) - All Features

**Dataset B includes all 21 features from the DEDUPLICATED data.**

This represents the "best case scenario" where we have access to all available information.

In [15]:
print("=" * 60)
print("STEP 5: CREATING DATASET B (FULL) - ALL FEATURES")
print("=" * 60)

# CRITICAL: Use df_deduplicated (not df!)
df_full = df_deduplicated.copy()

print(f"\n✅ Created Dataset B from deduplicated data")
print(f"Source: df_deduplicated ({len(df_deduplicated):,} rows)")
print(f"Result: df_full ({len(df_full):,} rows)")

# Verify
assert len(df_full) == len(df_deduplicated), "❌ ERROR: df_full has different size!"
print(f"\n✅ Verification passed: Same size as deduplicated data")

STEP 5: CREATING DATASET B (FULL) - ALL FEATURES

✅ Created Dataset B from deduplicated data
Source: df_deduplicated (229,781 rows)
Result: df_full (229,781 rows)

✅ Verification passed: Same size as deduplicated data


In [16]:
# Separate features and target for Dataset B
X_full = df_full.drop('Diabetes_012', axis=1)
y_full = df_full['Diabetes_012']

print("=== Dataset B (Full) ===")
print(f"Samples: {len(df_full):,}")
print(f"Features shape: {X_full.shape}")
print(f"Target shape: {y_full.shape}")
print(f"\nNumber of features: {X_full.shape[1]}")
print(f"\nFeature list:")
for i, col in enumerate(X_full.columns, 1):
    print(f"  {i:2d}. {col}")

=== Dataset B (Full) ===
Samples: 229,781
Features shape: (229781, 21)
Target shape: (229781,)

Number of features: 21

Feature list:
   1. HighBP
   2. HighChol
   3. CholCheck
   4. BMI
   5. Smoker
   6. Stroke
   7. HeartDiseaseorAttack
   8. PhysActivity
   9. Fruits
  10. Veggies
  11. HvyAlcoholConsump
  12. AnyHealthcare
  13. NoDocbcCost
  14. GenHlth
  15. MentHlth
  16. PhysHlth
  17. DiffWalk
  18. Sex
  19. Age
  20. Education
  21. Income


## 🧹 Step 6: Create Dataset A (Clean) - Remove Leaky Features

**Dataset A removes potentially leaky features from the DEDUPLICATED data.**

This represents a more realistic scenario for preventive screening.

In [17]:
print("=" * 60)
print("STEP 6: CREATING DATASET A (CLEAN) - REMOVE LEAKY FEATURES")
print("=" * 60)

print(f"\n🚫 Removing features: {potentially_leaky_features}")

# CRITICAL: Use df_deduplicated (not df!)
df_clean = df_deduplicated.drop(columns=potentially_leaky_features).copy()

print(f"\n✅ Created Dataset A from deduplicated data")
print(f"Source: df_deduplicated ({len(df_deduplicated):,} rows)")
print(f"Result: df_clean ({len(df_clean):,} rows)")

# Verify
assert len(df_clean) == len(df_deduplicated), "❌ ERROR: df_clean has different size!"
print(f"\n✅ Verification passed: Same size as deduplicated data")

STEP 6: CREATING DATASET A (CLEAN) - REMOVE LEAKY FEATURES

🚫 Removing features: ['DiffWalk', 'GenHlth', 'PhysHlth']

✅ Created Dataset A from deduplicated data
Source: df_deduplicated (229,781 rows)
Result: df_clean (229,781 rows)

✅ Verification passed: Same size as deduplicated data


In [18]:
# Separate features and target for Dataset A
X_clean = df_clean.drop('Diabetes_012', axis=1)
y_clean = df_clean['Diabetes_012']

print("=== Dataset A (Clean) ===")
print(f"Samples: {len(df_clean):,}")
print(f"Features shape: {X_clean.shape}")
print(f"Target shape: {y_clean.shape}")
print(f"\nNumber of features: {X_clean.shape[1]}")
print(f"\nRemoved features: {potentially_leaky_features}")
print(f"\nRemaining feature list:")
for i, col in enumerate(X_clean.columns, 1):
    print(f"  {i:2d}. {col}")

=== Dataset A (Clean) ===
Samples: 229,781
Features shape: (229781, 18)
Target shape: (229781,)

Number of features: 18

Removed features: ['DiffWalk', 'GenHlth', 'PhysHlth']

Remaining feature list:
   1. HighBP
   2. HighChol
   3. CholCheck
   4. BMI
   5. Smoker
   6. Stroke
   7. HeartDiseaseorAttack
   8. PhysActivity
   9. Fruits
  10. Veggies
  11. HvyAlcoholConsump
  12. AnyHealthcare
  13. NoDocbcCost
  14. MentHlth
  15. Sex
  16. Age
  17. Education
  18. Income


## ✅ Step 7: Verify Both Datasets Are Identical in Size

In [19]:
print("\n" + "=" * 60)
print("VERIFICATION: BOTH DATASETS SAME SIZE")
print("=" * 60)

print(f"\n📊 Dataset Comparison:")
print(f"Dataset B (Full):  {len(df_full):,} samples, {X_full.shape[1]} features")
print(f"Dataset A (Clean): {len(df_clean):,} samples, {X_clean.shape[1]} features")

# Critical checks
same_size = len(df_full) == len(df_clean)
same_target = y_full.equals(y_clean)

print(f"\n✅ Same sample size? {same_size}")
if not same_size:
    print(f"   ❌ ERROR: Dataset sizes don't match!")
    print(f"   Dataset B: {len(df_full):,}")
    print(f"   Dataset A: {len(df_clean):,}")
    print(f"   Difference: {abs(len(df_full) - len(df_clean)):,}")
    raise AssertionError("Dataset sizes must be identical!")

print(f"✅ Same target values? {same_target}")
if not same_target:
    print(f"   ❌ ERROR: Targets don't match!")
    raise AssertionError("Targets must be identical!")

print(f"\n📊 Target distribution (both datasets):")
print("-" * 60)
for cls, count in y_full.value_counts().sort_index().items():
    pct = (count / len(y_full)) * 100
    print(f"Class {int(cls)}: {count:6,} ({pct:5.2f}%)")

print("\n" + "=" * 60)
print("✅ BOTH DATASETS READY FOR MODELING")
print("=" * 60)
print(f"\nDataset B: {len(df_full):,} samples, {X_full.shape[1]} features (all features)")
print(f"Dataset A: {len(df_clean):,} samples, {X_clean.shape[1]} features (removed 3 leaky features)")
print(f"\n💡 Same sample size ensures fair model comparison!")


VERIFICATION: BOTH DATASETS SAME SIZE

📊 Dataset Comparison:
Dataset B (Full):  229,781 samples, 21 features
Dataset A (Clean): 229,781 samples, 18 features

✅ Same sample size? True
✅ Same target values? True

📊 Target distribution (both datasets):
------------------------------------------------------------
Class 0: 190,055 (82.71%)
Class 1:  4,629 ( 2.01%)
Class 2: 35,097 (15.27%)

✅ BOTH DATASETS READY FOR MODELING

Dataset B: 229,781 samples, 21 features (all features)
Dataset A: 229,781 samples, 18 features (removed 3 leaky features)

💡 Same sample size ensures fair model comparison!


## ⚖️ Step 8: Feature Scaling Strategy

**Why do we need scaling?**

Different features have different ranges:
- `BMI`: ranges from 14 to 98
- `Age`: ranges from 1 to 13
- Binary features: only 0 or 1

**Which algorithms need scaling?**
- ✅ **Need scaling:** Logistic Regression, SVM, KNN (distance-based)
- ❌ **Don't need scaling:** Random Forest, Decision Trees, XGBoost (tree-based)

**Our approach:**
- We'll use `StandardScaler` (mean=0, std=1)
- Apply it ONLY to continuous features: `BMI`, `MentHlth`, `PhysHlth` (if present)
- Leave binary/ordinal features as-is

**IMPORTANT:** We'll fit the scaler later (in training pipeline) to avoid data leakage!

In [20]:
# Identify feature types for scaling
print("=== Feature Types for Scaling ===")

# Continuous features that need scaling
continuous_features_full = ['BMI', 'MentHlth', 'PhysHlth']  # For Dataset B
continuous_features_clean = ['BMI', 'MentHlth']             # For Dataset A (PhysHlth removed)

print(f"\nDataset B (Full) - Continuous features to scale:")
for feat in continuous_features_full:
    if feat in X_full.columns:
        print(f"  - {feat}: range [{X_full[feat].min():.0f} - {X_full[feat].max():.0f}]")

print(f"\nDataset A (Clean) - Continuous features to scale:")
for feat in continuous_features_clean:
    if feat in X_clean.columns:
        print(f"  - {feat}: range [{X_clean[feat].min():.0f} - {X_clean[feat].max():.0f}]")

print(f"\n✅ We'll apply StandardScaler to these features in the modeling pipeline")

=== Feature Types for Scaling ===

Dataset B (Full) - Continuous features to scale:
  - BMI: range [12 - 98]
  - MentHlth: range [0 - 30]
  - PhysHlth: range [0 - 30]

Dataset A (Clean) - Continuous features to scale:
  - BMI: range [12 - 98]
  - MentHlth: range [0 - 30]

✅ We'll apply StandardScaler to these features in the modeling pipeline


## ⚖️ Step 8: Feature Scaling Strategy

**Why do we need scaling?**

Different features have different ranges:
- `BMI`: ranges from 14 to 98
- `Age`: ranges from 1 to 13
- Binary features: only 0 or 1

**Which algorithms need scaling?**
- ✅ **Need scaling:** Logistic Regression, SVM, KNN (distance-based)
- ❌ **Don't need scaling:** Random Forest, Decision Trees, XGBoost (tree-based)

**Our approach:**
- We'll use `StandardScaler` (mean=0, std=1)
- Apply it ONLY to continuous features: `BMI`, `MentHlth`, `PhysHlth` (if present)
- Leave binary/ordinal features as-is

**IMPORTANT:** We'll fit the scaler later (in training pipeline) to avoid data leakage!

In [21]:
# Identify feature types for scaling
print("=== Feature Types for Scaling ===")

# Continuous features that need scaling
continuous_features_full = ['BMI', 'MentHlth', 'PhysHlth']  # For Dataset B
continuous_features_clean = ['BMI', 'MentHlth']             # For Dataset A (PhysHlth removed)

print(f"\nDataset B (Full) - Continuous features to scale:")
for feat in continuous_features_full:
    if feat in X_full.columns:
        print(f"  - {feat}: range [{X_full[feat].min():.0f} - {X_full[feat].max():.0f}]")

print(f"\nDataset A (Clean) - Continuous features to scale:")
for feat in continuous_features_clean:
    if feat in X_clean.columns:
        print(f"  - {feat}: range [{X_clean[feat].min():.0f} - {X_clean[feat].max():.0f}]")

print(f"\n✅ We'll apply StandardScaler to these features in the modeling pipeline")

=== Feature Types for Scaling ===

Dataset B (Full) - Continuous features to scale:
  - BMI: range [12 - 98]
  - MentHlth: range [0 - 30]
  - PhysHlth: range [0 - 30]

Dataset A (Clean) - Continuous features to scale:
  - BMI: range [12 - 98]
  - MentHlth: range [0 - 30]

✅ We'll apply StandardScaler to these features in the modeling pipeline


## 💾 Step 9: Save Prepared Datasets

We'll save both datasets for use in future notebooks.

In [22]:
# Save Dataset B (Full)
df_full.to_csv('dataset_B_full.csv', index=False)
print("✅ Saved: dataset_B_full.csv")
print(f"   Shape: {df_full.shape}")
print(f"   Features: {df_full.shape[1] - 1} (+ 1 target)")

# Save Dataset A (Clean)
df_clean.to_csv('dataset_A_clean.csv', index=False)
print("\n✅ Saved: dataset_A_clean.csv")
print(f"   Shape: {df_clean.shape}")
print(f"   Features: {df_clean.shape[1] - 1} (+ 1 target)")

print("\n" + "="*60)
print("✅ Data preparation complete!")
print("="*60)

✅ Saved: dataset_B_full.csv
   Shape: (229781, 22)
   Features: 21 (+ 1 target)

✅ Saved: dataset_A_clean.csv
   Shape: (229781, 19)
   Features: 18 (+ 1 target)

✅ Data preparation complete!


## 📊 Step 8: Summary Comparison

In [23]:
# Create comparison table
comparison_data = {
    'Dataset': ['Dataset B (Full)', 'Dataset A (Clean)'],
    'Total Samples': [len(df_full), len(df_clean)],
    'Features': [X_full.shape[1], X_clean.shape[1]],
    'Removed Features': ['-', ', '.join(potentially_leaky_features)],
    'Use Case': ['Maximum predictive power', 'Realistic preventive screening']
}

comparison_df = pd.DataFrame(comparison_data)
print("\n=== Dataset Comparison ===")
print(comparison_df.to_string(index=False))


=== Dataset Comparison ===
          Dataset  Total Samples  Features            Removed Features                       Use Case
 Dataset B (Full)         229781        21                           -       Maximum predictive power
Dataset A (Clean)         229781        18 DiffWalk, GenHlth, PhysHlth Realistic preventive screening


In [24]:
## 🔍 VERIFICATION: Load and Inspect Both Datasets

import pandas as pd
import numpy as np

print("=" * 70)
print("LOADING AND VERIFYING SAVED DATASETS")
print("=" * 70)

# Load both datasets
df_full = pd.read_csv('C:\\Users\\yaros\\Desktop\\python\\faidm\\individual_project\\diabetes-classification-ml\\data_preparation\\dataset_B_full.csv')
df_clean = pd.read_csv('C:\\Users\\yaros\\Desktop\\python\\faidm\\individual_project\\diabetes-classification-ml\\data_preparation\\dataset_A_clean.csv')

print("\n✅ Both datasets loaded successfully!")

LOADING AND VERIFYING SAVED DATASETS

✅ Both datasets loaded successfully!


In [25]:
# 1. Basic shape comparison
print("\n" + "=" * 70)
print("1️⃣ SHAPE COMPARISON")
print("=" * 70)

print(f"\nDataset B (Full):")
print(f"  Rows: {len(df_full):,}")
print(f"  Columns: {len(df_full.columns)}")
print(f"  Total cells: {df_full.size:,}")

print(f"\nDataset A (Clean):")
print(f"  Rows: {len(df_clean):,}")
print(f"  Columns: {len(df_clean.columns)}")
print(f"  Total cells: {df_clean.size:,}")

print(f"\n📊 Comparison:")
print(f"  Same number of rows? {len(df_full) == len(df_clean)} ✅")
print(f"  Column difference: {len(df_full.columns) - len(df_clean.columns)} columns")


1️⃣ SHAPE COMPARISON

Dataset B (Full):
  Rows: 229,781
  Columns: 22
  Total cells: 5,055,182

Dataset A (Clean):
  Rows: 229,781
  Columns: 19
  Total cells: 4,365,839

📊 Comparison:
  Same number of rows? True ✅
  Column difference: 3 columns


In [26]:
# 2. Column comparison - What's in each dataset?
print("\n" + "=" * 70)
print("2️⃣ COLUMN COMPARISON")
print("=" * 70)

print(f"\n📋 Dataset B (Full) - {len(df_full.columns)} columns:")
print("-" * 70)
for i, col in enumerate(df_full.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\n📋 Dataset A (Clean) - {len(df_clean.columns)} columns:")
print("-" * 70)
for i, col in enumerate(df_clean.columns, 1):
    print(f"  {i:2d}. {col}")

# Find which columns are in B but not in A
columns_in_B_not_A = set(df_full.columns) - set(df_clean.columns)
columns_in_A_not_B = set(df_clean.columns) - set(df_full.columns)

print(f"\n🚫 Columns ONLY in Dataset B (removed in Dataset A):")
print("-" * 70)
if columns_in_B_not_A:
    for col in sorted(columns_in_B_not_A):
        print(f"  - {col}")
else:
    print("  None")

print(f"\n➕ Columns ONLY in Dataset A (not in Dataset B):")
print("-" * 70)
if columns_in_A_not_B:
    for col in sorted(columns_in_A_not_B):
        print(f"  - {col}")
else:
    print("  None (expected)")


2️⃣ COLUMN COMPARISON

📋 Dataset B (Full) - 22 columns:
----------------------------------------------------------------------
   1. Diabetes_012
   2. HighBP
   3. HighChol
   4. CholCheck
   5. BMI
   6. Smoker
   7. Stroke
   8. HeartDiseaseorAttack
   9. PhysActivity
  10. Fruits
  11. Veggies
  12. HvyAlcoholConsump
  13. AnyHealthcare
  14. NoDocbcCost
  15. GenHlth
  16. MentHlth
  17. PhysHlth
  18. DiffWalk
  19. Sex
  20. Age
  21. Education
  22. Income

📋 Dataset A (Clean) - 19 columns:
----------------------------------------------------------------------
   1. Diabetes_012
   2. HighBP
   3. HighChol
   4. CholCheck
   5. BMI
   6. Smoker
   7. Stroke
   8. HeartDiseaseorAttack
   9. PhysActivity
  10. Fruits
  11. Veggies
  12. HvyAlcoholConsump
  13. AnyHealthcare
  14. NoDocbcCost
  15. MentHlth
  16. Sex
  17. Age
  18. Education
  19. Income

🚫 Columns ONLY in Dataset B (removed in Dataset A):
----------------------------------------------------------------------
  

In [27]:
# 3. Target variable verification
print("\n" + "=" * 70)
print("3️⃣ TARGET VARIABLE VERIFICATION")
print("=" * 70)

print("\n📊 Dataset B (Full) - Target distribution:")
print("-" * 70)
for cls, count in df_full['Diabetes_012'].value_counts().sort_index().items():
    pct = (count / len(df_full)) * 100
    print(f"  Class {int(cls)}: {count:6,} ({pct:5.2f}%)")

print("\n📊 Dataset A (Clean) - Target distribution:")
print("-" * 70)
for cls, count in df_clean['Diabetes_012'].value_counts().sort_index().items():
    pct = (count / len(df_clean)) * 100
    print(f"  Class {int(cls)}: {count:6,} ({pct:5.2f}%)")

# Check if targets are identical
targets_identical = df_full['Diabetes_012'].equals(df_clean['Diabetes_012'])
print(f"\n✅ Target values identical? {targets_identical}")


3️⃣ TARGET VARIABLE VERIFICATION

📊 Dataset B (Full) - Target distribution:
----------------------------------------------------------------------
  Class 0: 190,055 (82.71%)
  Class 1:  4,629 ( 2.01%)
  Class 2: 35,097 (15.27%)

📊 Dataset A (Clean) - Target distribution:
----------------------------------------------------------------------
  Class 0: 190,055 (82.71%)
  Class 1:  4,629 ( 2.01%)
  Class 2: 35,097 (15.27%)

✅ Target values identical? True


In [28]:
# 4. Check common features - do they have same values?
print("\n" + "=" * 70)
print("4️⃣ COMMON FEATURES - VALUE COMPARISON")
print("=" * 70)

# Get columns that exist in both datasets
common_cols = list(set(df_full.columns) & set(df_clean.columns))
common_cols.remove('Diabetes_012')  # Already checked target
common_cols.sort()

print(f"\n📋 Checking {len(common_cols)} common features...")
print("-" * 70)

all_identical = True
for col in common_cols[:5]:  # Show first 5 as examples
    identical = df_full[col].equals(df_clean[col])
    status = "✅" if identical else "❌"
    print(f"{status} {col}: {'Identical' if identical else 'DIFFERENT'}")
    if not identical:
        all_identical = False

if len(common_cols) > 5:
    print(f"  ... and {len(common_cols) - 5} more features")

print(f"\n✅ All common features identical? {all_identical}")


4️⃣ COMMON FEATURES - VALUE COMPARISON

📋 Checking 18 common features...
----------------------------------------------------------------------
✅ Age: Identical
✅ AnyHealthcare: Identical
✅ BMI: Identical
✅ CholCheck: Identical
✅ Education: Identical
  ... and 13 more features

✅ All common features identical? True


In [29]:
# 5. Data types verification
print("\n" + "=" * 70)
print("5️⃣ DATA TYPES VERIFICATION")
print("=" * 70)

print("\n📊 Dataset B (Full):")
print(df_full.dtypes.value_counts())

print("\n📊 Dataset A (Clean):")
print(df_clean.dtypes.value_counts())

print("\n✅ Both datasets have only numeric types? ", end="")
numeric_only_B = all(df_full.dtypes.apply(lambda x: x.kind in 'iufc'))
numeric_only_A = all(df_clean.dtypes.apply(lambda x: x.kind in 'iufc'))
print(numeric_only_B and numeric_only_A)


5️⃣ DATA TYPES VERIFICATION

📊 Dataset B (Full):
float64    22
Name: count, dtype: int64

📊 Dataset A (Clean):
float64    19
Name: count, dtype: int64

✅ Both datasets have only numeric types? True


In [30]:
# 6. Missing values check
print("\n" + "=" * 70)
print("6️⃣ MISSING VALUES CHECK")
print("=" * 70)

missing_B = df_full.isnull().sum().sum()
missing_A = df_clean.isnull().sum().sum()

print(f"\nDataset B (Full): {missing_B} missing values")
print(f"Dataset A (Clean): {missing_A} missing values")

print(f"\n✅ No missing values in either dataset? {missing_B == 0 and missing_A == 0}")


6️⃣ MISSING VALUES CHECK

Dataset B (Full): 0 missing values
Dataset A (Clean): 0 missing values

✅ No missing values in either dataset? True


In [31]:
# 7. Sample rows comparison
print("\n" + "=" * 70)
print("7️⃣ SAMPLE ROWS (First 3 rows)")
print("=" * 70)

print("\n📋 Dataset B (Full) - First 3 rows:")
print("-" * 70)
print(df_full.head(3))

print("\n📋 Dataset A (Clean) - First 3 rows:")
print("-" * 70)
print(df_clean.head(3))


7️⃣ SAMPLE ROWS (First 3 rows)

📋 Dataset B (Full) - First 3 rows:
----------------------------------------------------------------------
   Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0           0.0     1.0       1.0        1.0  40.0     1.0     0.0   
1           0.0     0.0       0.0        0.0  25.0     1.0     0.0   
2           0.0     1.0       1.0        1.0  28.0     0.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  Veggies  HvyAlcoholConsump  \
0                   0.0           0.0     0.0      1.0                0.0   
1                   0.0           1.0     0.0      0.0                0.0   
2                   0.0           0.0     1.0      0.0                0.0   

   AnyHealthcare  NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex  \
0            1.0          0.0      5.0      18.0      15.0       1.0  0.0   
1            0.0          1.0      3.0       0.0       0.0       0.0  0.0   
2            1.0          1.0      5.0  

In [33]:
# 8. Final summary
print("\n" + "=" * 70)
print("✅ FINAL VERIFICATION SUMMARY")
print("=" * 70)

checks = {
    'Same number of rows': len(df_full) == len(df_clean),
    'Dataset B has 22 columns': len(df_full.columns) == 22,
    'Dataset A has 19 columns': len(df_clean.columns) == 19,
    'Target values identical': df_full['Diabetes_012'].equals(df_clean['Diabetes_012']),
    'Common features identical': all_identical,
    'No missing values': missing_B == 0 and missing_A == 0,
    'All numeric types': numeric_only_B and numeric_only_A,
    '3 features removed in A': len(columns_in_B_not_A) == 3,
}

print("\n📋 Verification Checklist:")
print("-" * 70)
for check, passed in checks.items():
    status = "✅" if passed else "❌"
    print(f"{status} {check}")

all_passed = all(checks.values())

print("\n" + "=" * 70)
if all_passed:
    print("🎉 ALL CHECKS PASSED - DATASETS ARE READY!")
    print("\nDataset B (Full):  229,781 rows × 22 cols (21 features + target)")
    print("Dataset A (Clean): 229,781 rows × 19 cols (18 features + target)")
    print("\nRemoved from Dataset A: DiffWalk, GenHlth, PhysHlth")
else:
    print("⚠️ SOME CHECKS FAILED - Review above")
print("=" * 70)



✅ FINAL VERIFICATION SUMMARY

📋 Verification Checklist:
----------------------------------------------------------------------
✅ Same number of rows
✅ Dataset B has 22 columns
✅ Dataset A has 19 columns
✅ Target values identical
✅ Common features identical
✅ No missing values
✅ All numeric types
✅ 3 features removed in A

🎉 ALL CHECKS PASSED - DATASETS ARE READY!

Dataset B (Full):  229,781 rows × 22 cols (21 features + target)
Dataset A (Clean): 229,781 rows × 19 cols (18 features + target)

Removed from Dataset A: DiffWalk, GenHlth, PhysHlth


---

## 🔍 Critical Analysis: Data Preparation Decisions

### **What We Did:**
1. Created two versions of the dataset
2. Identified potentially leaky features based on clinical reasoning
3. Prepared scaling strategy for distance-based algorithms
4. Kept data in raw form (no derived features yet)

### **Why We Made These Choices:**

#### **1. Two Datasets Approach**
**Rationale:**
- **Dataset B (Full)** allows us to see maximum achievable performance
- **Dataset A (Clean)** ensures our model works for real-world prevention
- Comparing them reveals the impact of potentially leaky features

**Theory (from lectures):**
- "Data Understanding" phase in CRISP-DM requires understanding causal relationships
- Target leakage occurs when features are consequences rather than causes
- Models with leakage may fail in production even with high test accuracy

#### **2. Features Identified as Potentially Leaky**

**`DiffWalk` (Difficulty Walking):**
- **Clinical reasoning:** Diabetic neuropathy causes nerve damage → difficulty walking
- **Risk:** High - this is a known complication of uncontrolled diabetes
- **Decision:** Remove in Dataset A

**`GenHlth` (General Health Rating):**
- **Clinical reasoning:** Self-reported health naturally decreases after diabetes diagnosis
- **Risk:** Medium - could be both cause and consequence
- **Decision:** Remove in Dataset A to be conservative

**`PhysHlth` (Days of Poor Physical Health):**
- **Clinical reasoning:** Similar to GenHlth - likely affected by diabetes symptoms
- **Risk:** Medium - measures consequences of disease
- **Decision:** Remove in Dataset A

#### **3. Why NOT Remove Other Features?**

**`Stroke` and `HeartDiseaseorAttack` - Why we kept them:**
- While diabetes increases cardiovascular risk, these can occur independently
- They represent comorbidities rather than direct consequences
- Removing them might hurt model performance without clear benefit
- If results show issues, we can revisit this decision

#### **4. No Derived Features (Yet)**
**Rationale:**
- Start simple - raw features first
- Tree-based models (Random Forest, XGBoost) can capture non-linearities automatically
- Feature engineering adds complexity - only worth it if baseline results are poor
- Easier to debug and interpret with original features

**Potential future features (if needed):**
- BMI categories (WHO standard: Underweight, Normal, Overweight, Obese)
- Age groups (Young, Middle-age, Senior)
- Interaction terms (e.g., Age × BMI)

### **Strengths of Our Approach:**
- ✅ **Transparent:** Clear documentation of which features removed and why
- ✅ **Scientific:** Based on clinical knowledge and causal reasoning
- ✅ **Flexible:** Can easily test both datasets and compare results
- ✅ **Practical:** Dataset A addresses real-world preventive screening use case
- ✅ **Simple:** No premature feature engineering

### **Limitations:**
- ⚠️ **Uncertainty:** We can't be 100% certain which features are truly leaky without domain expert validation
- ⚠️ **Trade-off:** Dataset A might have lower accuracy, but is more ethically sound for prevention
- ⚠️ **Binary decision:** We're either keeping or removing features - no "partial" use
- ⚠️ **Other potential leakage:** Features like `Stroke` or `HeartDiseaseorAttack` might also have some leakage

### **Implications for Model Development:**

**Expected outcomes:**
1. **Dataset B** will likely show higher accuracy (especially if leakage exists)
2. **Dataset A** will show more realistic performance for preventive screening
3. Large performance difference suggests significant leakage in removed features
4. Small performance difference validates our conservative feature removal

**Next steps:**
1. Exploratory analysis to understand feature relationships
2. Clustering to identify risk segments
3. Classification on BOTH datasets
4. Compare results and discuss implications

### **Ethical Considerations:**
- Using leaky features in production could lead to **false confidence** in predictions
- Healthcare systems need models that work for **early detection**, not just diagnosis confirmation
- Transparent documentation allows future researchers to make informed decisions
- Our two-dataset approach balances academic rigor with practical applicability

---

## ✅ Summary

**What we accomplished:**
- ✅ Created Dataset B (Full) - 21 features, all information
- ✅ Created Dataset A (Clean) - 18 features, removed potential leakage
- ✅ Prepared scaling strategy for modeling pipeline
- ✅ Saved both datasets for future analysis

**Ready for:**
- 📊 Notebook 03: Exploratory Analysis
- 🔵 Notebook 04: Clustering
- 🎯 Notebook 05: Classification

---